In [ ]:
# Ipython
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
from sympy import trigsimp


In [ ]:
from welib.yams.models.FTNSB_sympy import *
from welib.yams.models.OneRigidBody_sympy import get_model_one_body
from welib.yams.models.FTNSB_sympy_symbols import *
# --- Setup a wind turbine model based on predefined number of DOFs "FxTxRNA"
# NOTE: look at README.md for examples
print('>>> Setting up model...')
model = get_model('F0T1N0S1', mergeFndTwr=True, linRot=False,
                  yaw='zero', tilt='fixed',tiltShaft=True,
                  rot_elastic_type='Body', #rot_elastic_type='Body', 'Body' or 'SmallRot'
                  rot_elastic_smallAngle=False, # Keep False nu needs to be second order
                  orderMM=1, orderH=1,
                  twrDOFDir=['x','y','x','y'], # Order in which the flexible DOF of the tower are set
                 )
model.kdeqsSubs
model.coordinates
# --- Compute Kane's equations of motion
print('>>> Computing equations of motion...')
model.kaneEquations(Mform='TaylorExpanded')
# --- Perform additional substitution and small angle approximation
print('>>> Small angle approximation...')
extraSubs=model.shapeNormSubs
#extraSubs=[]
#extraSubs+=[(Symbol('u_xT1c'),1)]#extraSubs+=[(Symbol('v_yT1c'),0)]#extraSubs+=[(Symbol('J_xx_T'),0)]
#extraSubs+=[(Symbol('J_xx_N'),0)]#extraSubs+=[(Symbol('M_T'),0)]#extraSubs+=[(Symbol('x_BG'),0)]
print('Extra Subs:')
extraSubs
#print('Small angles:')
#model.smallAngles
#model.smallAngleApprox(model.smallAngles, extraSubs)
#model.smallAngleApproxEOM(model.smallAngles, extraSubs)
# --- Linearize equation of motions (with small angles)
#print('>>> Linearization...')
#model.linearize(noAcc=True, extraSubs=extraSubs)
#model.smallAngleLinearize(noAcc=True, extraSubs=extraSubs)
# --- Save as latex and python
#print('>>> Export...')
#model.smallAngleSaveTex(folder='./_tex',prefix='test',variables=['MM','FF','M','C','K','B'])
#model.savePython(folder='./_py',prefix='test',variables=['MM','FF','M','C','K','B'], extraSubs=extraSubs)

In [ ]:
#bd=model.body
#ref=model.ref
#bd.masscenter_pos_global.express(ref.frame)
twr = model.twr
twr
nac = model.nac
nac
rot = model.rot
rot
nac.masscenter_vel_inertial
#rot.masscenter_vel_inertial

In [ ]:
nac=model.nac
nac.masscenter_pos_global
nac.masscenter_vel_inertial
print('omega')
nac.omega_inertial.subs(extraSubs)
nac.R_b2g
omTilde=diff(nac.R_b2g,time)*(nac.R_b2g.T).simplify()
trigsimp(omTilde)

In [ ]:
from welib.yams.yams_sympy_tools import *
MM=model.kane.mass_matrix.subs(extraSubs)
#MM
MM2=smallAngleApprox(MM, model.twr.vcList,2)
MM2=smallAngleApprox(MM2, [theta_tilt],1)
MM2=smallAngleApprox(MM, [phi_y],1)
MM2.simplify()
MM2.expand()
#DM=(MM-MM2).expand()
#DM


In [ ]:
MM2[0,0].simplify().expand()
MM2[1,1].simplify().expand()

In [ ]:
MM2[0,0].expand()
MM2[1,1].expand()
#MM[1,1]

In [ ]:
ff = model.kane.forcing.subs(extraSubs).subs(model.kdeqsSubs)
ff.simplify()
ff[1]
#ff2=smallAngleApprox(ff, twr.vcList, 2)
#ff2=smallAngleApprox(ff2, [theta_tilt], 1)
#ff2=smallAngleApprox(ff2, [phi_y],1)
#ff2.expand()
#Df=(ff-ff2).expand()
#Df

In [ ]:
#model.K

In [ ]:
model.kane.mass_matrix.subs(extraSubs)

In [ ]:
# --- Full system mass matrix
model.kane.mass_matrix.subs(extraSubs)

In [ ]:
# --- Small angle approximation mass matrix
model._sa_mass_matrix

In [ ]:
model.kane.forcing.subs(extraSubs).subs(model.kdeqsSubs)

In [ ]:
# --- Full system Forcing  RHS)
model.kane.forcing.subs(extraSubs).subs(model.kdeqsSubs)

In [ ]:
# --- Small angle approximation forcing
model._sa_forcing

In [ ]:
# --- Equation of motions
#EOM=(model.fr+model.frstar).subs(model.kdeqsSubs)

In [ ]:
# --- Small angle Equation of motions
#model.smallAngleApproxEOM(model.smallAngles, extraSubs)
#model.EOM


In [ ]:
# --- Linearization of equation of motions
M,C,K,B = model.linearize(EOM=model._sa_EOM)

M


In [ ]:
# --- display some info about bodies
nac=model.nac
twr=model.twr
print(nac)
print(twr)
#twr.bodyMassMatrix()